<a href="https://colab.research.google.com/github/BebeShen/colab/blob/main/DL_StockPredict_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 認識Prophet

Prophet 是一個由 Facebook Core Data Science Team 發表的開源代碼庫，用於時間序列預測，基於 Python 和 R 語言。


## 安裝Prophet以及相關套件

>`!`是為了讓colab執行如同terminal執行如同terminal


有些文章可能會引導安裝的package為`fbprophet`，根據[此篇issue](https://github.com/facebook/prophet/issues/1874)可以知道，`fbprophet`已經改名為`prophet`了

In [ ]:
!pip install quandl matplotlib numpy prophet pystan pandas pytrends twstock requests

測試prophet是否安裝成功

In [6]:
import prophet
print(prophet.__version__)

1.0


## 讀取股價資訊

利用`twstock`套件讀取股票資訊



In [ ]:
import twstock
# 抓取台積電(2330)的近31日資料
# (預設為抓取30天)
stock = twstock.Stock('2330')
# # 擷取自 2022 年 2 月至今之資料
# stock.fetch_from(2022, 3)
stock.price

## 建立模型進行預測

In [11]:
def Get_Stock_Informations(stock_code, start_date, stop_date):
    information_url = ('http://140.116.86.242:8081/stock/' +
                       'api/v1/api_get_stock_info_from_date_json/' +
                       str(stock_code) + '/' +
                       str(start_date) + '/' +
                       str(stop_date)
                       )
    result = requests.get(information_url).json()
    if(result['result'] == 'success'):
        return result['data']
    return dict([])

In [31]:
import matplotlib.pyplot as plt
import prophet

model = prophet.Prophet()

# stock_to_predict = twstock.Stock('2492')
# stock_to_predict.fetch_from(2022, 1)
# stock_to_predict.close

stocks = Get_Stock_Informations(2492, 20220101, 20220410)
close = list(x['close'] for x in stocks)


[136,
 135.5,
 139.5,
 141,
 142,
 143.5,
 141,
 140,
 141,
 144,
 144,
 145,
 145,
 144,
 144,
 144.5,
 136,
 138.5,
 141.5,
 141.5,
 142.5,
 139.5,
 138,
 142,
 148,
 150,
 151,
 151,
 146,
 146.5,
 151.5,
 153.5,
 157.5,
 157,
 158.5,
 158.5,
 155,
 159.5,
 158.5,
 158.5,
 156.5,
 153.5,
 150.5,
 151,
 154,
 155.5,
 158.5,
 156.5,
 157.5,
 157,
 154.5,
 154.5,
 156,
 160.5,
 167.5,
 167,
 171.5,
 173,
 177,
 171]

## 串接交易API

In [4]:
import random
import requests
from datetime import datetime
import time

# 取得股票資訊
# Input:
#   stock_code: 股票ID
#   start_date: 開始日期，YYYYMMDD
#   stop_date: 結束日期，YYYYMMDD
# Output: 持有股票陣列

def Get_Stock_Informations(stock_code, start_date, stop_date):
    information_url = ('http://140.116.86.242:8081/stock/' +
                       'api/v1/api_get_stock_info_from_date_json/' +
                       str(stock_code) + '/' +
                       str(start_date) + '/' +
                       str(stop_date)
                       )
    result = requests.get(information_url).json()
    if(result['result'] == 'success'):
        return result['data']
    return dict([])

# 取得持有股票
# Input:
#   account: 使用者帳號
#   password: 使用者密碼
# Output: 持有股票陣列

def Get_User_Stocks(account, password):
    data = {
      'account': account,
      'password': password
    }
    search_url = 'http://140.116.86.242:8081/stock/api/v1/get_user_stocks'
    result = requests.post(search_url, data=data).json()
    if(result['result'] == 'success'):
        return result['data']
    return dict([])

# 交易紀錄
def Transcation_History(account, start_date, end_date):
    information_url = ('http://140.116.86.242:8081/stock/' +
                       'api/v1/transcation_history/' +
                       str(start_date) + '/' +
                       str(end_date) + '/' +
                       str(account)
                       )
    result = requests.get(information_url).json()
    if(result['result'] == 'success'):
        return result['data']
    return dict([])

# 預約購入股票
# Input:
#   account: 使用者帳號
#   password: 使用者密碼
#   stock_code: 股票ID
#   stock_shares: 購入張數
#   stock_price: 購入價格
# Output: 是否成功預約購入(True/False)

def Buy_Stock(account, password, stock_code, stock_shares, stock_price):
    print('Buying stock...')
    data = {'account': account,
            'password': password,
            'stock_code': stock_code,
            'stock_shares': stock_shares,
            'stock_price': stock_price}
    buy_url = 'http://140.116.86.242:8081/stock/api/v1/buy'
    result = requests.post(buy_url, data=data).json()
    print('Result: ' + result['result'] + "\nStatus: " + result['status'])
    return result['result'] == 'success'

# 預約售出股票
# Input:
#   account: 使用者帳號
#   password: 使用者密碼
#   stock_code: 股票ID
#   stock_shares: 售出張數
#   stock_price: 售出價格
# Output: 是否成功預約售出(True/False)

def Sell_Stock(account, password, stock_code, stock_shares, stock_price):
    print('Selling stock...')
    data = {'account': account,
            'password': password,
            'stock_code': stock_code,
            'stock_shares': stock_shares,
            'stock_price': stock_price}
    sell_url = 'http://140.116.86.242:8081/stock/api/v1/sell'
    result = requests.post(sell_url, data=data).json()
    print('Result: ' + result['result'] + "\nStatus: " + result['status'])
    return result['result'] == 'success'

Get_User_Stocks("F74072235", "F74072235")

[]